# Comparador de coches

Mostrar en gráficos de radar las características más importantes para poder comparar 2 coches. Usa una función de escalado de datos para poder hacer la comparación entre diferentes escalas. 

**Extra: Añadir la primera foto del coche.**



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import base64

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded = base64.b64encode(image_file.read()).decode()
    return f"data:image/png;base64,{encoded}"


In [4]:
df = pd.read_csv("../data/coches_consolidado_limpio.csv")

In [5]:
df_comparar: pd.DataFrame = df[["modelo_titulo","consumo_medio","precio", "potencia", "kilometraje","ruta_imagen"]]

print(len(df_comparar))

coche_a = df_comparar.iloc[2]
coche_a = coche_a.fillna(0)

coche_b = df_comparar.iloc[33]
coche_b =coche_b.fillna(0)


17518


In [10]:
'''quitamos las variables modelo_titulo y ruta_imagen ya que no son utiles para comparar las características'''
categorias = coche_a[['consumo_medio', 'precio', 'potencia', 'kilometraje']].keys()
valores_a = coche_a[categorias].values
valores_b = coche_b[categorias].values
imagen_a = encode_image(f"../{coche_a['ruta_imagen']}")
imagen_b = encode_image(f"../{coche_b['ruta_imagen']}")

scaler = StandardScaler()
valores_a = scaler.fit_transform(valores_a.reshape(-1,1))
valores_b = scaler.fit_transform(valores_b.reshape(-1,1))

df_comparar = pd.DataFrame({coche_a['modelo_titulo'] :valores_a.flatten(),
                            coche_b['modelo_titulo'] :valores_b.flatten()},
                            index=categorias).reset_index().rename(columns={'index':'Característica'})


df_comparar = pd.melt(df_comparar, id_vars=['Característica'], var_name= "Modelo", value_name="Valor")
print(df_comparar)

  Característica                            Modelo     Valor
0  consumo_medio  C5 X 1.6 Puretech Shine EAT8 180 -0.955410
1         precio  C5 X 1.6 Puretech Shine EAT8 180  0.506667
2       potencia  C5 X 1.6 Puretech Shine EAT8 180 -0.943917
3    kilometraje  C5 X 1.6 Puretech Shine EAT8 180  1.392660
4  consumo_medio         Polo 1.0 TSI Advance 70kW -0.662588
5         precio         Polo 1.0 TSI Advance 70kW -0.398969
6       potencia         Polo 1.0 TSI Advance 70kW -0.660513
7    kilometraje         Polo 1.0 TSI Advance 70kW  1.722070


In [12]:


fig = px.line_polar(df_comparar, r="Valor", theta="Característica", color="Modelo",
                    line_close=True)#, title="Comparación de Características de Coches")

fig.update_traces(fill='toself')
fig.update_layout(
                polar=dict(
                    radialaxis=dict(
                        visible=True,
                        tickfont=dict(color="black", size=14), 
                        range=[-1, max(df_comparar['Valor']) + 0.1]
                    )
                ),
                legend=dict(
                        orientation="h",  # Pone la leyenda en una sola fila horizontal
                        yanchor="bottom",  # Ancla la leyenda al fondo
                        y=1.05,  # Coloca la leyenda por encima del gráfico
                        xanchor="center",  # Centra la leyenda
                        x=0.5,  # Centra la leyenda horizontalmente
                        font=dict(size=12)  # Tamaño de fuente más pequeño para la leyenda
                    ),
                    margin=dict(t=0, b=0, l=0, r=0)  # Elimina márgenes para centrar mejor el gráfico
                )

fig.add_layout_image(
    dict(
        source= imagen_a,
        x=-0.3, y=0.5,
        xref="paper", yref="paper",
        sizex=0.4, sizey=0.6,
        xanchor="right", yanchor="middle",
        layer="below"
    )
)

fig.add_layout_image(
    dict(
        source=imagen_b,
        x=1.25, y=0.5,
        xref="paper", yref="paper",
        sizex=0.6, sizey=0.8,
        xanchor="left", yanchor="middle",
        layer="below"
    )
)

fig.update_layout(
    annotations=[
        dict(
            x=-0.9, y=0.7,  
            xref="paper", yref="paper",
            text=coche_a['modelo_titulo'],
            showarrow=False,
            font=dict(size=14, color="black")
        ),
        dict(
            x=1.9, y=0.7,  
            xref="paper", yref="paper",
            text=coche_b['modelo_titulo'],
            showarrow=False,
            font=dict(size=14, color="black")
        )
    ],
    margin=dict(l=300, r=300), 
    width=900, height=600 
)

fig.show()
